In [1]:
import pandas as pd
import numpy as np
import datetime as datetime
from arise_prototype import capacity_check as cc

In [2]:
date_columns = ['DATUM', 'START', 'ENDE', 'P1_START', 'P1_ENDE', 'P2_START', 'P2_ENDE', 'P3_START', 'P3_ENDE']
df_Schichtplan = pd.read_csv("../data/Kapazitätsplanung-20220121_Werksplanung.csv", parse_dates=date_columns)
date_columns = ['DATUM', 'MSTART', 'MENDE', 'MP1_START', 'MP1_ENDE', 'MP2_START', 'MP2_ENDE', 'MP3_START', 'MP3_ENDE']
df_Maschinenplan = pd.read_csv("../data/Kapazitätsplanung-20220121_Maschinenplanung.csv", parse_dates=date_columns)

# Kalkulieren der Produktionsdaten

In [3]:
# Daten laden
df = pd.read_csv("../data/Auftragsfolgen-20211207.csv")
df['LTermin'] = pd.to_datetime(df['LTermin'], format='%Y-%m-%d %H:%M:%S')
df['LTermin'] = df['LTermin'].dt.to_pydatetime()
df = df[df['ID_Maschstatus']==1]
df = df[['MaschNr', 'Laufzeit_Soll', 'LTermin', 'KndNr', 'Suchname', 'AKNR', 'ArtNr_Teil', 'TeilNr', 'SchrittNr']]
df = df.drop_duplicates()
start = '2021-05-15 00:00:00'
end = '2021-05-20 00:00:00'
mask = (df['LTermin']>=start) & (df['LTermin']<end)
df = df.loc[mask]


In [4]:
df = cc.calculate_production_date(df, df_Schichtplan)
df

,MaschNr,Laufzeit_Soll,LTermin,KndNr,Suchname,AKNR,ArtNr_Teil,TeilNr,SchrittNr,Production_date
78476,Handarbeit f. Handelsware,4,2021-05-17,14979,TVILUM,118145,29600,1,1,2021-05-14
78483,SL 5,9,2021-05-17,11722,MUECHER,118205,20783,1,1,2021-05-14
78624,Handarbeit f. Handelsware,30,2021-05-17,11464,RS GUIDESYSTEMS,117933,31965,1,1,2021-05-14
79084,SL 9 ROTA,27,2021-05-18,10883,BETTE,117702,24502,1,1,2021-05-17
79107,SL 9 ROTA,15,2021-05-18,10883,BETTE,117704,24503,1,1,2021-05-17
...,...,...,...,...,...,...,...,...,...,...
511662,SL 5,75,2021-05-17,12381,LEFELD,118305,32445,1,1,2021-05-14
511669,SL 5,10,2021-05-17,12403,KOETTER,117740,31681,1,1,2021-05-14
511674,SL 5,28,2021-05-17,12376,GUNDLACH,119245,19929,1,1,2021-05-14
511691,SL 5,40,2021-05-17,13074,KOECKERLING,118468,23918,1,1,2021-05-14


# Benötigte Kapazität für einen Tag

In [5]:
df =  cc.parse_machine_number(df)
df

,MaschNr,Laufzeit_Soll,LTermin,KndNr,Suchname,AKNR,ArtNr_Teil,TeilNr,SchrittNr,Production_date,machine,machine_id
78483,SL 5,9,2021-05-17,11722,MUECHER,118205,20783,1,1,2021-05-14,SL 5,5
79084,SL 9 ROTA,27,2021-05-18,10883,BETTE,117702,24502,1,1,2021-05-17,SL 9,9
79107,SL 9 ROTA,15,2021-05-18,10883,BETTE,117704,24503,1,1,2021-05-17,SL 9,9
130347,SL 2 Rota,780,2021-05-18,13907,CARTO RAAD,117096,19137,1,1,2021-05-17,SL 2,2
130354,SL 2,54,2021-05-18,10495,KAIMANN,118322,10219,1,2,2021-05-17,SL 2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
511662,SL 5,75,2021-05-17,12381,LEFELD,118305,32445,1,1,2021-05-14,SL 5,5
511669,SL 5,10,2021-05-17,12403,KOETTER,117740,31681,1,1,2021-05-14,SL 5,5
511674,SL 5,28,2021-05-17,12376,GUNDLACH,119245,19929,1,1,2021-05-14,SL 5,5
511691,SL 5,40,2021-05-17,13074,KOECKERLING,118468,23918,1,1,2021-05-14,SL 5,5


In [6]:
df_workload = cc.calculate_machine_workload(df)
df_workload

,machine_id,date,workload
0,5,2021-05-14,932
1,9,2021-05-14,467
2,2,2021-05-14,586
3,4,2021-05-14,774
4,8,2021-05-14,247
5,6,2021-05-14,2931
6,10,2021-05-14,75
7,5,2021-05-17,22
8,9,2021-05-17,79
9,2,2021-05-17,1210


# Schichtplan und Maschinenplan laden 

In [7]:
df_capacity = cc.get_capacity(df_workload['machine_id'].unique(), df_workload['date'].unique(), df_Schichtplan, df_Maschinenplan)
df_workload_capacity = df_workload.merge(df_capacity, on=['machine_id', 'date'])
df_workload_capacity

TODO: Check if date and machine combination is unique in            Maschinenschichten


,machine_id,date,workload,capacity
0,5,2021-05-14,932,600
1,9,2021-05-14,467,600
2,2,2021-05-14,586,959
3,4,2021-05-14,774,600
4,8,2021-05-14,247,600
5,6,2021-05-14,2931,1019
6,10,2021-05-14,75,600
7,5,2021-05-17,22,750
8,9,2021-05-17,79,750
9,2,2021-05-17,1210,750


# Vergleich der Kapazität mit der Produktionszeit

In [23]:
def compare_capacity(df):
    for index, row in df.iterrows():
        if int(row['workload']) > int(row['capacity']):
            print('Kapazität an SL ', row['machine_id'], ' am', row['date'], ' überschritten.')
            print('Kapazität: ', row['capacity'])
            print('Produktionszeit für geplante Aufträge: ', row['workload'])


In [24]:
compare_capacity(df_workload_capacity)

Kapazität an SL  5  am 2021-05-14 00:00:00  überschritten.
Kapazität:  600
Produktionszeit für geplante Aufträge:  932
Kapazität an SL  4  am 2021-05-14 00:00:00  überschritten.
Kapazität:  600
Produktionszeit für geplante Aufträge:  774
Kapazität an SL  6  am 2021-05-14 00:00:00  überschritten.
Kapazität:  1019
Produktionszeit für geplante Aufträge:  2931
Kapazität an SL  2  am 2021-05-17 00:00:00  überschritten.
Kapazität:  750
Produktionszeit für geplante Aufträge:  1210
